<a href="https://colab.research.google.com/github/anslava00/PySpark_sber/blob/main/PySpark_lesson2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark findspark nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re 
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import multiprocessing
import matplotlib.pyplot as plt
import os

conf = SparkConf().set('spark.ui.port', '4050').set('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')\
                  .set('spark.dynamicAllocation.enabled', 'true')\
                  .set('spark.shuffle.service.enabled', 'true') #трекер, чтобы возвращать ресурсы
sc = SparkContext(conf=conf)
spark = SparkSession.builder.master('local[*]').getOrCreate()

# DZ 1

In [ ]:
def movie_maping(x):
    return x.split('\t')[1: 3]

def movie_filter(x):
    try:
        if float(x[1]) > 4:
            return True
        else:
            return False
    except:
        return False
    return True


In [ ]:
text_rdd = sc.textFile('user_ratedmovies.dat')\
                      .map(movie_maping)\
                      .filter(movie_filter)\
                      .groupByKey().mapValues(len)\
                      .sortBy(lambda x: x[1], ascending=False)
text_rdd.take(10)

[('2571', 900),
 ('318', 887),
 ('296', 878),
 ('2959', 828),
 ('4993', 756),
 ('7153', 719),
 ('5952', 697),
 ('858', 690),
 ('50', 688),
 ('2858', 680)]

In [ ]:
text_rdd.getNumPartitions()

2

In [ ]:
import pandas as pd
nltk.download('stopwords')
from nltk.corpus import stopwords
from scipy import spatial

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
w2v = pd.read_pickle('w2v_vectors.pickle')

In [ ]:
stop_words = stopwords.words("russian")
stop_words = set(stop_words)

In [ ]:
for i in range(len(w2v) - 1, -1, -1):
    if w2v[i][0] in stop_words:
        del w2v[i]

In [ ]:
for index, tup in enumerate(w2v):
    if index%10 != 0 :
        print(tup[0], end=' ')
    else:
        print(tup[0], )

это
всё знать раскольник свой сказать говорить человек весь стать мочь
который очень соня рука петрович дело разумихин хотеть иванович ваш
минута видеть смотреть время пойти слово думать лицо глаз самый
дверь казаться катерина комната голова прийти свидригайлов порфирий день раскольников
взять дуня первый тотчас пётр спросить иметь стоять дом идти
деньга понимать выйти заметить романович мысль квартира год оба точно
бог хе сделать последний подумать брат мать час несколько начать
лестница должный посмотреть жить прямо друг ребёнок любить пульхерия случай
место александрович вид господин действительно дать семёнович войти сердце совершенно
вопрос проговорить вчера взгляд помнить сидеть особенно авдотья шаг сторона
весьма хотя стол жизнь родя именно продолжать улица слишком право
понять отвечать наш узнать некоторый черта убить новый пора сестра
ходить немой сегодня лужин пройти таки родион всякий скорый дунёчек
почему прибавить слышать никто ах странный угол делать голос каждый
бояться 

In [ ]:
print(w2v)

[('это', [0.14321378, -0.80575335, -0.49581036, -0.06394294, -1.2238435, -0.44410643, 0.49273458, 0.17389631, -0.27776885, 0.50880307, -0.07283105, 0.72964376, 1.1526197, 0.94315845, -1.0720603, -0.235505, 1.2034348, -0.3744024, -0.36203885, -0.07914155, 0.3772516, -0.878622, -0.927112, 0.73365724, 0.107223585, -0.39829564, 0.61899257, 0.29151887, 1.013499, 0.03111156]), ('всё', [-0.30770057, -1.296609, -0.7222447, -0.20211408, -0.7666911, -1.0031737, -0.26696622, -0.0668253, 0.0026058692, -0.105746254, 0.16301736, 0.41609567, 0.6745765, 1.1453478, -0.99610287, -0.53095675, 1.4379495, -0.48172295, -0.14234278, -0.49253845, 0.28369433, -0.6198971, -1.158686, 0.1808683, -0.5055639, -0.3938737, -0.076981455, 0.6512924, 1.5167931, 0.015842797]), ('знать', [0.19544418, -0.7807339, 0.06808824, -0.14120847, -2.031599, -0.87786776, -0.13631094, 0.12638684, -0.3159053, 0.7993867, -0.5642886, 0.7992176, 0.7489015, 1.1367697, -0.28135785, -0.5803256, 1.405251, -0.69162613, 0.1689085, 0.16315201, 

In [ ]:
wv_rdd = sc.parallelize(w2v)

In [ ]:
def wv_maping(all_word):
    def _wv_maping(word):
        word_dict = {}
        for tup in all_word:
            if tup != word:
                result = 1 - spatial.distance.cosine(word[1], tup[1])
                word_dict[tup[0]] = result
        word_dict = sorted(word_dict.items(), key=lambda x: x[1], reverse=True)[:10]
        return word[0], word_dict 
    return _wv_maping

In [ ]:
synon_dict = wv_rdd.map(wv_maping(wv_rdd.collect()))
synon_dict

PythonRDD[14] at RDD at PythonRDD.scala:53

In [ ]:
synon_dict.take(2)

[('это',
  [('вздор', 0.9239644408226013),
   ('равно', 0.8888441324234009),
   ('ясно', 0.8856370449066162),
   ('правда', 0.8584197759628296),
   ('лгать', 0.8425521850585938),
   ('думать', 0.8386391997337341),
   ('действительно', 0.837155818939209),
   ('видеть', 0.8360357880592346),
   ('относительно', 0.8360316753387451),
   ('всё', 0.8356289863586426)]),
 ('всё',
  [('вздор', 0.8659670948982239),
   ('равно', 0.8550696969032288),
   ('это', 0.8356289863586426),
   ('рассказать', 0.8059406280517578),
   ('последний', 0.8053026795387268),
   ('правда', 0.8028042912483215),
   ('сообразить', 0.8017109036445618),
   ('утро', 0.7971288561820984),
   ('неужели', 0.7915520668029785),
   ('ясно', 0.7872923016548157)])]

In [ ]:
axe_money = synon_dict.filter(lambda sword: (sword[0] == 'топор') | (sword[0] == 'деньга'))

In [ ]:
axe_money.take(2)

[('деньга',
  [('вещь', 0.9063147902488708),
   ('отдать', 0.8706483840942383),
   ('убить', 0.8420271873474121),
   ('значит', 0.8386284708976746),
   ('известно', 0.8266708254814148),
   ('отец', 0.8146180510520935),
   ('получить', 0.8137907385826111),
   ('ограбить', 0.8044242262840271),
   ('копейка', 0.8012078404426575),
   ('полечка', 0.7804852724075317)]),
 ('топор',
  [('левый', 0.9236524105072021),
   ('старуха', 0.922432005405426),
   ('колено', 0.9206660389900208),
   ('побежать', 0.9120596051216125),
   ('ломать', 0.9102391004562378),
   ('бросить', 0.9063428640365601),
   ('схватить', 0.9024109840393066),
   ('обнять', 0.8999080061912537),
   ('бить', 0.8985461592674255),
   ('положить', 0.8972359895706177)])]

In [ ]:
axe_money.collect()

[('деньга',
  [('вещь', 0.9063147902488708),
   ('отдать', 0.8706483840942383),
   ('убить', 0.8420271873474121),
   ('значит', 0.8386284708976746),
   ('известно', 0.8266708254814148),
   ('отец', 0.8146180510520935),
   ('получить', 0.8137907385826111),
   ('ограбить', 0.8044242262840271),
   ('копейка', 0.8012078404426575),
   ('полечка', 0.7804852724075317)]),
 ('топор',
  [('левый', 0.9236524105072021),
   ('старуха', 0.922432005405426),
   ('колено', 0.9206660389900208),
   ('побежать', 0.9120596051216125),
   ('ломать', 0.9102391004562378),
   ('бросить', 0.9063428640365601),
   ('схватить', 0.9024109840393066),
   ('обнять', 0.8999080061912537),
   ('бить', 0.8985461592674255),
   ('положить', 0.8972359895706177)])]